In [1]:
from datetime import *
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud.exceptions import NotFound
import pandas as pd
import numpy as np
import yahoo_fin.stock_info as si
from utils.utils_bigquery import *

In [2]:
# Ruta credenciales de CGP
key_path = key_path
project = project_id

# Sources
table = 'bronze_ticker_info'
dataset = 'bronze'
table_conca = f'{project}.{dataset}.{table}'

# Loading tables
table_to_save = 'silver_ticker_info'
dataset_to_save = 'silver'
table_conca_to_save = f'{project}.{dataset_to_save}.{table_to_save}'


In [3]:
# Conectamos con Bigquery
bigquery = BigQueryUtils(key_path)

In [4]:
# Nos traemos las columnas que creemos son mas relevantes
df = bigquery.run_query(
    f"""
    SELECT
        ticker,
        shortName,
        industry,
        sector,
        recommendationKey,
        fullTimeEmployees,
        marketCap,
        enterpriseValue,
        floatShares,
        sharesOutstanding,
        sharesShort,
        shortRatio,
        bookValue,
        currentPrice,
        totalCash,
        totalCashPerShare,
        totalDebt,
        totalRevenue,
        revenuePerShare,
        returnOnAssets,
        returnOnEquity,
        operatingCashflow,
        operatingMargins
    FROM {project}.{dataset}.{table}
    """
)
df

,ticker,shortName,industry,sector,recommendationKey,fullTimeEmployees,marketCap,enterpriseValue,floatShares,sharesOutstanding,...,currentPrice,totalCash,totalCashPerShare,totalDebt,totalRevenue,revenuePerShare,returnOnAssets,returnOnEquity,operatingCashflow,operatingMargins
0,LUNRW,None,None,None,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AXS^E,None,None,None,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,SHO^H,None,None,None,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MET^A,None,None,None,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,XFLT^A,None,None,None,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7115,RDY,Dr. Reddy's Laboratories Ltd,Drug Manufacturers—Specialty & Generic,Healthcare,buy,27048.0,1.381583e+10,-5.022766e+10,119949319.0,166575008.0,...,82.4850,9.462600e+10,568.169,3.067500e+10,2.885070e+11,289.40433,0.10928,0.20589,4.267900e+10,0.23391
7116,INCR,Intercure Ltd.,Drug Manufacturers—Specialty & Generic,Healthcare,buy,270.0,9.570266e+07,1.929151e+08,24852155.0,45572700.0,...,2.1000,1.013260e+08,2.223,1.956770e+08,3.555530e+08,7.80200,-0.00108,-0.12985,-5.363100e+07,-0.09039
7117,BGM,Qilian International Holding Gr,Drug Manufacturers—Specialty & Generic,Healthcare,none,298.0,4.205812e+07,1.806209e+07,1612611.0,4826480.0,...,5.8200,2.535528e+07,3.546,1.007560e+05,2.987046e+07,4.17800,-0.03835,-0.16522,2.955899e+06,-0.05406
7118,SXTC,"China SXT Pharmaceuticals, Inc.",Drug Manufacturers—Specialty & Generic,Healthcare,none,75.0,2.964062e+06,-5.455423e+06,3471885.0,3476090.0,...,0.8527,1.207034e+07,3.472,3.695005e+06,1.928497e+06,2.22300,-0.04965,-0.21652,-1.928053e+06,7.33808


In [5]:
tickers_indicators = bigquery.run_query(
    f"""
    SELECT DISTINCT
        ticker
    FROM {project}.silver.silver_indicators
    """
)
tickers_indicators

,ticker
0,CMG
1,NTAP
2,CVX
3,LNT
4,SWKS
...,...
472,GRMN
473,IFF
474,INCY
475,TPR


In [6]:
# Hacemos un INNER para traernos solo los ticker de los que hemos trabajado los indicadores
df_filtered = pd.merge(df, tickers_indicators, how='inner', on='ticker')
df_filtered = df_filtered[df.columns]
df_filtered

,ticker,shortName,industry,sector,recommendationKey,fullTimeEmployees,marketCap,enterpriseValue,floatShares,sharesOutstanding,...,currentPrice,totalCash,totalCashPerShare,totalDebt,totalRevenue,revenuePerShare,returnOnAssets,returnOnEquity,operatingCashflow,operatingMargins
0,NEM,Newmont Corporation,Gold,Basic Materials,buy,21700.0,6.014828e+10,6.672165e+10,1.144641e+09,1.147430e+09,...,52.4200,2.652000e+09,2.311,9.225000e+09,1.487500e+10,14.576,0.02785,-0.08089,3.823000e+09,0.30532
1,FSLR,"First Solar, Inc.",Solar,Technology,buy,6700.0,2.458442e+10,2.384386e+10,1.011968e+08,1.070470e+08,...,229.6600,1.740343e+09,16.258,6.219340e+08,3.764233e+09,35.211,0.07989,0.18106,1.187273e+09,0.36907
2,ENPH,"Enphase Energy, Inc.",Solar,Technology,buy,3157.0,1.611988e+10,1.627635e+10,1.310968e+08,1.354220e+08,...,119.0344,1.646404e+09,12.158,1.321347e+09,1.420449e+09,10.449,0.01854,0.13958,3.575680e+08,0.00979
3,STLD,"Steel Dynamics, Inc.",Steel,Basic Materials,hold,12600.0,1.860740e+10,2.070709e+10,1.439303e+08,1.543030e+08,...,120.5900,1.248347e+09,8.090,3.028444e+09,1.814712e+10,113.044,0.10815,0.22783,2.716117e+09,0.12069
4,NUE,Nucor Corporation,Steel,Basic Materials,buy,32000.0,3.532302e+10,3.836032e+10,2.353705e+08,2.373380e+08,...,148.8300,5.434158e+09,22.896,7.040368e+09,3.269452e+10,133.720,0.08581,0.17630,5.929265e+09,0.10979
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
472,TER,"Teradyne, Inc.",Semiconductor Equipment & Materials,Technology,buy,6200.0,2.185743e+10,2.132202e+10,1.625022e+08,1.631760e+08,...,133.9500,4.605580e+08,2.822,8.016300e+07,2.704030e+09,17.518,0.09140,0.19190,6.464750e+08,0.21220
473,NRG,"NRG Energy, Inc.",Utilities—Independent Power Producers,Utilities,buy,18131.0,1.719888e+10,2.846431e+10,1.933511e+08,2.063800e+08,...,83.3360,3.960000e+08,1.919,1.092600e+10,2.884100e+10,132.602,0.05429,0.59641,2.130000e+09,0.21415
474,VTRS,Viatris Inc.,Drug Manufacturers—Specialty & Generic,Healthcare,hold,33000.0,1.396418e+10,3.037627e+10,1.188902e+09,1.193520e+09,...,11.7000,1.019200e+09,0.854,1.737160e+10,1.523920e+10,12.737,0.02535,-0.03203,2.297000e+09,0.09495
475,CTLT,"Catalent, Inc.",Drug Manufacturers—Specialty & Generic,Healthcare,hold,18000.0,1.076469e+10,1.556915e+10,1.798542e+08,1.809800e+08,...,59.4800,1.620000e+08,0.895,4.990000e+09,4.135000e+09,22.751,-0.00712,-0.28290,2.500000e+08,0.00559


In [7]:
# Nos quedamos con solo una fila por ticker

# Reemplazar valores NaN en la columna 'market' con 0
df_filtered['marketCap'].fillna(0, inplace=True)

# Encontrar el índice de la fila con el valor más alto en 'market' para cada ticker
idx = df_filtered.groupby('ticker')['marketCap'].idxmax()

# Seleccionar esas filas del DataFrame original
df_filtered = df_filtered.loc[idx]

# Eliminamos los tickers que el marketcap sea 0
df_filtered = df_filtered[df_filtered['marketCap'] != 0]

df_filtered

,ticker,shortName,industry,sector,recommendationKey,fullTimeEmployees,marketCap,enterpriseValue,floatShares,sharesOutstanding,...,currentPrice,totalCash,totalCashPerShare,totalDebt,totalRevenue,revenuePerShare,returnOnAssets,returnOnEquity,operatingCashflow,operatingMargins
274,A,"Agilent Technologies, Inc.",Diagnostics & Research,Healthcare,buy,18000.0,4.037208e+10,4.154804e+10,2.863853e+08,2.875300e+08,...,140.410,1.779000e+09,6.187,2.932000e+09,6.497000e+09,22.269,0.08906,0.24658,1.786000e+09,0.21103
13,AAL,"American Airlines Group, Inc.",Airlines,Industrials,hold,137400.0,6.583457e+09,3.716495e+10,6.479106e+08,6.567040e+08,...,10.025,8.987000e+09,13.685,3.944700e+10,5.344800e+10,81.548,0.02779,NaN,2.015000e+09,0.09823
250,AAPL,Apple Inc.,Consumer Electronics,Technology,buy,161000.0,3.455132e+12,3.493579e+12,1.517981e+10,1.520410e+10,...,227.250,6.180100e+10,4.065,1.013040e+11,3.856030e+11,24.957,0.22612,1.60583,1.130410e+11,0.29556
353,ABBV,AbbVie Inc.,Drug Manufacturers—General,Healthcare,buy,50000.0,3.460260e+11,4.065709e+11,1.762882e+09,1.766340e+09,...,195.900,1.315700e+10,7.449,7.093800e+10,5.500000e+10,31.100,0.07936,0.54262,1.863800e+10,0.34560
112,ABT,Abbott Laboratories,Medical Devices,Healthcare,buy,114000.0,1.951559e+11,2.039545e+11,1.728936e+09,1.739900e+09,...,112.165,7.219000e+09,4.149,1.484500e+10,4.072500e+10,23.402,0.06255,0.14441,7.900000e+09,0.17664
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
396,XYL,Xylem Inc.,Specialty Industrial Machinery,Industrials,buy,23000.0,3.294601e+10,3.439016e+10,2.411076e+08,2.428930e+08,...,135.640,8.150000e+08,3.355,2.104000e+09,8.396000e+09,34.747,0.04155,0.07553,1.205000e+09,0.12679
27,YUM,"Yum! Brands, Inc.",Restaurants,Consumer Cyclical,buy,35000.0,3.763815e+10,4.943522e+10,2.805268e+08,2.811650e+08,...,133.865,5.200000e+08,1.849,1.201200e+10,7.105000e+09,25.240,0.25167,NaN,1.630000e+09,0.35961
117,ZBH,"Zimmer Biomet Holdings, Inc.",Medical Devices,Healthcare,hold,18000.0,2.337518e+10,2.896757e+10,2.031290e+08,2.036520e+08,...,114.780,4.201000e+08,2.063,6.034600e+09,7.524800e+09,36.369,0.04544,0.07946,1.523400e+09,0.20705
299,ZBRA,Zebra Technologies Corporation,Communication Equipment,Technology,buy,9750.0,1.774056e+10,1.987798e+10,5.117598e+07,5.158040e+07,...,343.940,4.110000e+08,7.968,2.355000e+09,4.357000e+09,84.762,0.04023,0.07326,5.190000e+08,0.14051


In [8]:
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 477 entries, 274 to 476
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ticker             477 non-null    object 
 1   shortName          477 non-null    object 
 2   industry           477 non-null    object 
 3   sector             477 non-null    object 
 4   recommendationKey  477 non-null    object 
 5   fullTimeEmployees  470 non-null    float64
 6   marketCap          477 non-null    float64
 7   enterpriseValue    473 non-null    float64
 8   floatShares        477 non-null    float64
 9   sharesOutstanding  477 non-null    float64
 10  sharesShort        477 non-null    float64
 11  shortRatio         477 non-null    float64
 12  bookValue          477 non-null    float64
 13  currentPrice       477 non-null    float64
 14  totalCash          473 non-null    float64
 15  totalCashPerShare  473 non-null    float64
 16  totalDebt          471 non-nu

In [9]:
# Seleccionar solo las columnas numéricas
numerical_columns = df_filtered.select_dtypes(include=[np.number]).columns

# Identificar columnas numéricas con valores NaN
columns_with_nans = df_filtered[numerical_columns].columns[df_filtered[numerical_columns].isna().any()]

# Calcular la mediana para cada columna numérica con valores NaN
medianas = df_filtered[columns_with_nans].median()

# Rellenar los valores NaN en las columnas numéricas con la mediana correspondiente
df_filtered[columns_with_nans] = df_filtered[columns_with_nans].fillna(medianas)

df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 477 entries, 274 to 476
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ticker             477 non-null    object 
 1   shortName          477 non-null    object 
 2   industry           477 non-null    object 
 3   sector             477 non-null    object 
 4   recommendationKey  477 non-null    object 
 5   fullTimeEmployees  477 non-null    float64
 6   marketCap          477 non-null    float64
 7   enterpriseValue    477 non-null    float64
 8   floatShares        477 non-null    float64
 9   sharesOutstanding  477 non-null    float64
 10  sharesShort        477 non-null    float64
 11  shortRatio         477 non-null    float64
 12  bookValue          477 non-null    float64
 13  currentPrice       477 non-null    float64
 14  totalCash          477 non-null    float64
 15  totalCashPerShare  477 non-null    float64
 16  totalDebt          477 non-nu

In [10]:
# Define sector groupings
sector_groups = {
    'Technology': 'Technology and Communications',
    'Communication Services': 'Technology and Communications',
    'Financial Services': 'Financials and Real Estate',
    'Real Estate': 'Financials and Real Estate',
    'Healthcare': 'Healthcare and Defensive Products',
    'Consumer Defensive': 'Healthcare and Defensive Products',
    'Industrials': 'Industrials and Basic Materials',
    'Basic Materials': 'Industrials and Basic Materials',
    'Consumer Cyclical': 'Consumer, Energy, and Utilities',
    'Energy': 'Consumer, Energy, and Utilities',
    'Utilities': 'Consumer, Energy, and Utilities'
}

# Add new columns to the DataFrame
df['sector_group'] = df['sector'].map(sector_groups)

# Show the updated DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7120 entries, 0 to 7119
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ticker             7120 non-null   object 
 1   shortName          6562 non-null   object 
 2   industry           6362 non-null   object 
 3   sector             6362 non-null   object 
 4   recommendationKey  6450 non-null   object 
 5   fullTimeEmployees  5285 non-null   float64
 6   marketCap          5991 non-null   float64
 7   enterpriseValue    5839 non-null   float64
 8   floatShares        5928 non-null   float64
 9   sharesOutstanding  5928 non-null   float64
 10  sharesShort        6041 non-null   float64
 11  shortRatio         6020 non-null   float64
 12  bookValue          6291 non-null   float64
 13  currentPrice       6450 non-null   float64
 14  totalCash          6165 non-null   float64
 15  totalCashPerShare  6080 non-null   float64
 16  totalDebt          5992 

In [11]:
import pandas as pd

# Suponiendo que 'df' es tu DataFrame y contiene una columna 'Industry' con las industrias
# Aquí se definen los grupos de industrias reducidos

industry_groups = {
    'Technology and Communications': [
        'Computer Hardware', 'Communication Equipment', 'Consumer Electronics',
        'Electronic Components', 'Software—Application', 'Software—Infrastructure',
        'Information Technology Services', 'Telecom Services',
        'Internet Content & Information','Semiconductors', 'Semiconductor Equipment & Materials'
    ],
    'Health and Biotechnology': [
        'Biotechnology', 'Medical Devices', 'Medical Care Facilities',
        'Medical Distribution', 'Medical Instruments & Supplies',
        'Diagnostics & Research', 'Drug Manufacturers—General',
        'Drug Manufacturers—Specialty & Generic', 'Pharmaceutical Retailers',
        'Healthcare Plans', 'Health Information Services'

    ],
    'Energy and Natural Resources': [
        'Gold', 'Copper', 'Oil & Gas E&P', 'Oil & Gas Equipment & Services',
        'Oil & Gas Integrated', 'Oil & Gas Midstream', 'Oil & Gas Refining & Marketing',
        'Solar', 'Steel', 'Agricultural Inputs'
    ],
    'Financial Services and Banking': [
        'Banks—Diversified', 'Banks—Regional', 'Capital Markets',
        'Credit Services', 'Asset Management', 'Financial Data & Stock Exchanges',
        'Insurance Brokers', 'Insurance—Diversified', 'Insurance—Life',
        'Insurance—Property & Casualty', 'Insurance—Reinsurance', 'Insurance—Specialty'
    ],
    'Retail and Consumer Goods': [
        'Apparel Manufacturing', 'Apparel Retail', 'Auto & Truck Dealerships',
        'Auto Manufacturers', 'Auto Parts', 'Discount Stores', 'Grocery Stores',
        'Internet Retail', 'Packaged Foods', 'Restaurants', 'Specialty Retail',
        'Home Improvement Retail', 'Footwear & Accessories', 'Lodging', 'Leisure',
        'Travel Services', 'Personal Services', 'Luxury Goods', 'Confectioners',
        'Beverages—Brewers', 'Beverages—Non-Alcoholic', 'Beverages—Wineries & Distilleries',
        'Airlines', 'Trucking', 'Railroads', 'Resorts & Casinos', 'Food Distribution', 'Tobacco',
        'Entertainment', 'Household & Personal Products'
    ],
    'Real Estate and Infrastructure': [
        'Building Materials', 'Building Products & Equipment', 'Engineering & Construction',
        'Real Estate Services', 'REIT—Office', 'REIT—Retail', 'REIT—Residential',
        'REIT—Healthcare Facilities', 'REIT—Industrial', 'REIT—Hotel & Motel',
        'REIT—Specialty', 'REIT—Diversified', 'Residential Construction', 'REIT - Healthcare Facilities',
        'Farm & Heavy Construction Machinery'
    ],
    'Industry and Utilities': [
        'Aerospace & Defense', 'Specialty Chemicals', 'Chemicals', 'Specialty Industrial Machinery',
        'Utilities—Diversified', 'Farm Products', 'Utilities—Renewable',
        'Utilities—Independent Power Producers', 'Utilities—Regulated Electric',
        'Utilities—Regulated Gas', 'Utilities—Regulated Water', 'Waste Management',
        'Packaging & Containers', 'Tools & Accessories', 'Furnishings, Fixtures & Appliances',
        'Electronic Gaming & Multimedia', 'Integrated Freight & Logistics',
        'Scientific & Technical Instruments', 'Advertising Agencies', 'Consulting Services',
        'Security & Protection Services', 'Rental & Leasing Services', 'Pollution & Treatment Controls',
        'Electrical Equipment & Parts', 'Specialty Business Services', 'Industrial Distribution', 'Conglomerates'
    ]
}

# Crea un diccionario para mapear cada industria a su grupo
industry_to_group = {industry: group for group, industries in industry_groups.items() for industry in industries}

# Asigna la nueva columna al DataFrame
df['industry_group'] = df['industry'].map(industry_to_group)

# Imprime el DataFrame para verificar
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7120 entries, 0 to 7119
Data columns (total 25 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ticker             7120 non-null   object 
 1   shortName          6562 non-null   object 
 2   industry           6362 non-null   object 
 3   sector             6362 non-null   object 
 4   recommendationKey  6450 non-null   object 
 5   fullTimeEmployees  5285 non-null   float64
 6   marketCap          5991 non-null   float64
 7   enterpriseValue    5839 non-null   float64
 8   floatShares        5928 non-null   float64
 9   sharesOutstanding  5928 non-null   float64
 10  sharesShort        6041 non-null   float64
 11  shortRatio         6020 non-null   float64
 12  bookValue          6291 non-null   float64
 13  currentPrice       6450 non-null   float64
 14  totalCash          6165 non-null   float64
 15  totalCashPerShare  6080 non-null   float64
 16  totalDebt          5992 

In [15]:
try:
    # Filtramos solamente los nuevos registros
    df_incremental = bigquery.select_for_incremental(id='ticker', table=table_conca_to_save, new_df=df_filtered)

    if not df_incremental.empty:
        # Guardamos los datos en Bigquery
        bigquery.save_dataframe(df_incremental, project, dataset_to_save, table_to_save, if_exists='append', schema=None)
        print(f'New records loaded')
    else:
        print('No new records to load.')

# En el caso de no tener datos en Bigquery, guardamos todo el df
except Exception as e:
    bigquery.save_dataframe(df, project, dataset_to_save, table_to_save, if_exists='replace', schema=None)
    print('New data persisted')
    print(f'Exception encountered: {e}')

No new records to load.
